# 第4章 Unicode文本和字节序列

## 4.1 字符问题

- 字符的标识：码点，是0\~1114111范围内的数（十进制），在Unicode标准中以4\~6个十六进制数表示，前面加“U+”，取值范围是U+0000\~U+10FFFF。
- 字符的具体描述取决于所用的编码。编码是在码点和字节序列之间转换时使用的算法。

In [6]:
# 编码和解码
s = 'café'
len(s)

4

In [7]:
b = s.encode('utf8')
b, len(b)

(b'caf\xc3\xa9', 5)

In [8]:
b.decode('utf8')

'café'

## 4.2 字节概要

In [9]:
# 包含5个字节的bytes和bytearray对象
cafe = bytes('café', encoding='utf_8')
cafe

b'caf\xc3\xa9'

In [10]:
cafe[0], cafe[:1]

(99, b'c')

In [11]:
cafe_arr = bytearray(cafe)
cafe_arr, cafe_arr[-1:]

(bytearray(b'caf\xc3\xa9'), bytearray(b'\xa9'))

## 4.3 处理编码和解码问题

1. 处理UnicodeEncodeError

In [12]:
# 把str编码成字节序列，有些成功，有些需要处理错误
city = 'Sāo Paulo'
city.encode('utf_8')

b'S\xc4\x81o Paulo'

In [13]:
city.encode('utf_16')

b'\xff\xfeS\x00\x01\x01o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [27]:
city.encode('iso8859_1', errors='ignore')

b'So Paulo'

In [19]:
city.encode('cp437', errors='replace')

b'S?o Paulo'

In [20]:
city.encode('cp437', errors='xmlcharrefreplace')

b'S&#257;o Paulo'

2. 处理UnicodeDecodeError

In [22]:
# 把字节序列解码成str，有些成功，有些需要处理错误
octets = b'Montr\xe9al'
octets.decode('cp1252')

'Montréal'

In [23]:
octets.decode('iso8859_7')

'Montrιal'

In [28]:
octets.decode('koi8_r')

'MontrИal'

In [29]:
octets.decode('utf_8', errors='replace')

'Montr�al'

## 4.4 默认编码

In [30]:
import locale
import sys

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
    """

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(f'{expression:>30} -> {value!r}')

 locale.getpreferredencoding() -> 'cp936'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'cp936'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'gbk'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


## 4.5 为了正确比较而规范化Unicode字符

- NFC（Normalization Form C）：使用最少的码点构成等价的字符串。
- NFD：把合成字符分解成基字符和单独的组合字符。

In [31]:
from unicodedata import normalize

In [32]:
s1 = 'café'
s2 = 'cafe\N{COMBINING ACUTE ACCENT}'
len(s1), len(s2)

(4, 5)

In [33]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [34]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

## 4.6 极端“规范化”：去掉变音符

In [38]:
import unicodedata


def shave_marks(txt):
    """删除所有变音符"""
    # 把所有字符分解成基字符和组合记号
    norm_txt = unicodedata.normalize('NFD', txt)
    # 过滤所有组合记号
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))
    # 重组所有字符
    return unicodedata.normalize('NFC', shaved)

In [39]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [40]:
greek = 'Ζέφυρος, Zéfiro'
shave_marks(greek)

'Ζεφυρος, Zefiro'

In [47]:
import string


def shave_marks_latin(txt):
    """删除所有拉丁基字符上的变音符"""
    norm_txt = unicodedata.normalize('NFD', txt)
    latin_base = False
    preserve = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:
            continue  # 忽略拉丁基字符的变音符
        preserve.append(c)
        # 如果不是组合字符，那就是新的基字符
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
    shaved = ''.join(preserve)
    return unicodedata.normalize('NFC', shaved)

In [48]:
single_map = str.maketrans("""‚ƒ„ˆ‹‘’“”•–—˜›""",  # <1>
                           """'f"^<''""---~>""")

multi_map = str.maketrans({  # <2>
    '€': 'EUR',
    '…': '...',
    'Æ': 'AE',
    'æ': 'ae',
    'Œ': 'OE',
    'œ': 'oe',
    '™': '(TM)',
    '‰': '<per mille>',
    '†': '**',
    '‡': '***',
})

multi_map.update(single_map)  # <3>


def dewinize(txt):
    """把cp1252符号替换为ASCII字符或字符序列"""
    return txt.translate(multi_map)

def asciize(txt):
    # 去掉变音符
    no_marks = shave_marks_latin(dewinize(txt))
    no_marks = no_marks.replace('ß', 'ss')
    # 使用NFKC规范化形式把字符和码点组合起来
    return unicodedata.normalize('NFKC', no_marks)

In [49]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'

In [50]:
dewinize(order)

'"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."'

In [51]:
asciize(order)

'"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."'

## 4.7 Unicode文本排序

In [53]:
# 巴西产水果的列表排序
import locale
my_locale = locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')

fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits, key=locale.strxfrm)

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

## 4.8 Unicode数据库

Unicode标准提供了一个完整的数据库，不仅包括码点与字符名称之间的映射表，还包括各个字符的元数据，以及字符之间的关系。

- unicodedata.name()：返回一个字符在标准中的官方名称
- unicodedata.numeric()：返回一个字符在标准中的数值

## 4.9 支持str和bytes的双模式API

1. 正则表达式中的str和bytes

In [55]:
import re

# str类型
re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
# bytes类型
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
            " as 1729 = 1³ + 12³ = 9³ + 10³.")

# bytes正则表达式只能搜索bytes字符串
text_bytes = text_str.encode('utf_8')

print(f'Text\n  {text_str!r}')
print('Numbers')
# str模式r'\d+'只能匹配泰米尔数值和ASCII数字
print('  str  :', re_numbers_str.findall(text_str)) 
# bytes模式rb'\d+'只能匹配ASCII字节中的数字
print('  bytes:', re_numbers_bytes.findall(text_bytes))  
print('Words')
# str模式r'\w+'能匹配字母、上标、泰米尔数字和ASCII数字
print('  str  :', re_words_str.findall(text_str))  
# bytes模式rb'\w+'只能匹配ASCII字节中的字母和数字
print('  bytes:', re_words_bytes.findall(text_bytes)) 

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


2. os函数中的str和bytes

- `os`模块中所有接收文件名或路径名的函数，既可以传入`str`参数，也可以传入`bytes`参数。
- 传入`str`参数时，使用`sys.getfilesystemencoding()`获得的编码解码器自动转换参数，操作系统回显时也使用编码解码器进行解码。
- `os`模块提供了特殊的编码解码函数`os.fsencode(name_or_path)`和`os.fsdecode(name_or_path)`。